#**스마트폰 센서 데이터 기반 모션 분류**
# 단계2 : 기본 모델링


## 0.미션

* 데이터 전처리
    * 가변수화, 데이터 분할, NaN 확인 및 조치, 스케일링 등 필요한 전처리 수행
* 다양한 알고리즘으로 분류 모델 생성
    * 최소 4개 이상의 알고리즘을 적용하여 모델링 수행
    * 성능 비교
    * 각 모델의 성능을 저장하는 별도 데이터 프레임을 만들고 비교
* 옵션 : 다음 사항은 선택사항입니다. 시간이 허용하는 범위 내에서 수행하세요.
    * 상위 N개 변수를 선정하여 모델링 및 성능 비교
        * 모델링에 항상 모든 변수가 필요한 것은 아닙니다.
        * 변수 중요도 상위 N개를 선정하여 모델링하고 타 모델과 성능을 비교하세요.
        * 상위 N개를 선택하는 방법은, 변수를 하나씩 늘려가며 모델링 및 성능 검증을 수행하여 적절한 지점을 찾는 것입니다.

## 1.환경설정

### (1) 라이브러리 불러오기

* 세부 요구사항
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
    - 필요하다고 판단되는 라이브러리를 추가하세요.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 필요하다고 판단되는 라이브러리를 추가하세요.
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from sklearn.preprocessing import MinMaxScaler



* 함수 생성

In [2]:
# 변수의 특성 중요도 계산하기
def plot_feature_importance(importance, names, result_only = False, topn = 'all'):
    feature_importance = np.array(importance)
    feature_name = np.array(names)

    data={'feature_name':feature_name,'feature_importance':feature_importance}
    fi_temp = pd.DataFrame(data)

    #변수의 특성 중요도 순으로 정렬하기
    fi_temp.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    fi_temp.reset_index(drop=True, inplace = True)

    if topn == 'all' :
        fi_df = fi_temp.copy()
    else :
        fi_df = fi_temp.iloc[:topn]

    #변수의 특성 중요도 그래프로 그리기
    if result_only == False :
        plt.figure(figsize=(10,20))
        sns.barplot(x='feature_importance', y='feature_name', data = fi_df)

        plt.xlabel('importance')
        plt.ylabel('feature name')
        plt.grid()

    return fi_df

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### (2) 데이터 불러오기

* 주어진 데이터셋
    * data01_train.csv : 학습 및 검증용
* 세부 요구사항
    - 전체 데이터 'data01_train.csv' 를 불러와 'data' 이름으로 저장합니다.
        - data에서 변수 subject는 삭제합니다.
    - 데이터프레임에 대한 기본 정보를 확인합니다.( .head(), .shape 등)

#### 1) 데이터 로딩

In [4]:
import joblib
data_path = '/content/drive/MyDrive/2023.10.25_미니프로젝트5차_실습자료밀 데이터/데이터/data01_train.csv'
fi_path = '/content/drive/MyDrive/2023.10.25_미니프로젝트5차_실습자료밀 데이터/데이터/feature_importances.pkl'
data = pd.read_csv(data_path)
fi = joblib.load(fi_path)

In [5]:
data.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288508,-0.009196,-0.103362,-0.988986,-0.962797,-0.967422,-0.989000,-0.962596,-0.965650,-0.929747,...,-0.816696,-0.042494,-0.044218,0.307873,0.072790,-0.601120,0.331298,0.165163,21,STANDING
1,0.265757,-0.016576,-0.098163,-0.989551,-0.994636,-0.987435,-0.990189,-0.993870,-0.987558,-0.937337,...,-0.693515,-0.062899,0.388459,-0.765014,0.771524,0.345205,-0.769186,-0.147944,15,LAYING
2,0.278709,-0.014511,-0.108717,-0.997720,-0.981088,-0.994008,-0.997934,-0.982187,-0.995017,-0.942584,...,-0.829311,0.000265,-0.525022,-0.891875,0.021528,-0.833564,0.202434,-0.032755,11,STANDING
3,0.289795,-0.035536,-0.150354,-0.231727,-0.006412,-0.338117,-0.273557,0.014245,-0.347916,0.008288,...,-0.408956,-0.255125,0.612804,0.747381,-0.072944,-0.695819,0.287154,0.111388,17,WALKING
4,0.394807,0.034098,0.091229,0.088489,-0.106636,-0.388502,-0.010469,-0.109680,-0.346372,0.584131,...,-0.563437,-0.044344,-0.845268,-0.974650,-0.887846,-0.705029,0.264952,0.137758,17,WALKING_DOWNSTAIRS


In [6]:
data.drop('subject', axis=1, inplace=True)

#### 2) 기본 정보 조회

In [8]:
data.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity
0,0.288508,-0.009196,-0.103362,-0.988986,-0.962797,-0.967422,-0.989000,-0.962596,-0.965650,-0.929747,...,-0.487737,-0.816696,-0.042494,-0.044218,0.307873,0.072790,-0.601120,0.331298,0.165163,STANDING
1,0.265757,-0.016576,-0.098163,-0.989551,-0.994636,-0.987435,-0.990189,-0.993870,-0.987558,-0.937337,...,-0.237820,-0.693515,-0.062899,0.388459,-0.765014,0.771524,0.345205,-0.769186,-0.147944,LAYING
2,0.278709,-0.014511,-0.108717,-0.997720,-0.981088,-0.994008,-0.997934,-0.982187,-0.995017,-0.942584,...,-0.535287,-0.829311,0.000265,-0.525022,-0.891875,0.021528,-0.833564,0.202434,-0.032755,STANDING
3,0.289795,-0.035536,-0.150354,-0.231727,-0.006412,-0.338117,-0.273557,0.014245,-0.347916,0.008288,...,-0.004012,-0.408956,-0.255125,0.612804,0.747381,-0.072944,-0.695819,0.287154,0.111388,WALKING
4,0.394807,0.034098,0.091229,0.088489,-0.106636,-0.388502,-0.010469,-0.109680,-0.346372,0.584131,...,-0.157832,-0.563437,-0.044344,-0.845268,-0.974650,-0.887846,-0.705029,0.264952,0.137758,WALKING_DOWNSTAIRS


In [9]:
data.shape

(5881, 562)

In [11]:
data.describe()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
count,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,...,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000
mean,0.274811,-0.017799,-0.109396,-0.603138,-0.509815,-0.604058,-0.628151,-0.525944,-0.605374,-0.465490,...,0.126955,-0.305883,-0.623548,0.008524,-0.001185,0.009340,-0.007099,-0.491501,0.059299,-0.054594
std,0.067614,0.039422,0.058373,0.448807,0.501815,0.417319,0.424345,0.485115,0.413043,0.544995,...,0.249176,0.322808,0.310371,0.339730,0.447197,0.608190,0.476738,0.509069,0.297340,0.278479
min,-0.503823,-0.684893,-1.000000,-1.000000,-0.999844,-0.999667,-1.000000,-0.999419,-1.000000,-1.000000,...,-0.965725,-0.979261,-0.999765,-0.976580,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-0.980143
25%,0.262919,-0.024877,-0.121051,-0.992774,-0.977680,-0.980127,-0.993602,-0.977865,-0.980112,-0.936067,...,-0.021610,-0.541969,-0.845985,-0.122361,-0.294369,-0.481718,-0.373345,-0.811397,-0.018203,-0.141555
50%,0.277154,-0.017221,-0.108781,-0.943933,-0.844575,-0.856352,-0.948501,-0.849266,-0.849896,-0.878729,...,0.133887,-0.342923,-0.712677,0.010278,0.005146,0.011448,-0.000847,-0.709441,0.182893,0.003951
75%,0.288526,-0.010920,-0.098163,-0.242130,-0.034499,-0.262690,-0.291138,-0.068857,-0.268539,-0.013690,...,0.288944,-0.127371,-0.501158,0.154985,0.285030,0.499857,0.356236,-0.511330,0.248435,0.111932
max,1.000000,1.000000,1.000000,1.000000,0.916238,1.000000,1.000000,0.967664,1.000000,1.000000,...,0.946700,0.989538,0.956845,1.000000,1.000000,0.998702,0.996078,0.977344,0.478157,1.000000


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5881 entries, 0 to 5880
Columns: 562 entries, tBodyAcc-mean()-X to Activity
dtypes: float64(561), object(1)
memory usage: 25.2+ MB


## **2. 데이터 전처리**

* 가변수화, 데이터 분할, NaN 확인 및 조치, 스케일링 등 필요한 전처리를 수행한다.


### (1) 데이터 분할1 : x, y

* 세부 요구사항
    - x, y로 분할합니다.

In [14]:
x = data.drop('Activity', axis=1)
y = data.loc[:, 'Activity']

### (2) 스케일링(필요시)


* 세부 요구사항
    - 스케일링을 필요로 하는 알고리즘 사용을 위해서 코드 수행
    - min-max 방식 혹은 standard 방식 중 한가지 사용.

In [16]:
scaler = MinMaxScaler()
x_train_s = scaler.fit_transform(x_train)
x_val_s = scaler.transform(x_val)

### (3) 데이터분할2 : train, validation

* 세부 요구사항
    - train : val = 8 : 2 혹은 7 : 3
    - random_state 옵션을 사용하여 다른 모델과 비교를 위해 성능이 재현되도록 합니다.

In [15]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

## **3. 기본 모델링**



* 세부 요구사항
    - 최소 4개 이상의 알고리즘을 적용하여 모델링을 수행한다.
    - 각 알고리즘별로 전체 변수로 모델링, 상위 N개 변수를 선택하여 모델링을 수행하고 성능 비교를 한다.
    - (옵션) 알고리즘 중 1~2개에 대해서, 변수 중요도 상위 N개를 선정하여 모델링하고 타 모델과 성능을 비교.
        * 상위 N개를 선택하는 방법은, 변수를 하나씩 늘려가며 모델링 및 성능 검증을 수행하여 적절한 지점을 찾는 것이다.

### (1) 알고리즘1 :

In [18]:
rf = RandomForestClassifier()
rf.fit(x_train, y_train)



RandomForestClassifier()

In [19]:
y_pred = rf.predict(x_val)
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))
print('Accuracy :', accuracy_score(y_val, y_pred))

                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       223
           SITTING       0.95      0.96      0.95       206
          STANDING       0.97      0.95      0.96       218
           WALKING       0.99      0.98      0.98       200
WALKING_DOWNSTAIRS       0.98      0.97      0.97       158
  WALKING_UPSTAIRS       0.97      1.00      0.99       172

          accuracy                           0.98      1177
         macro avg       0.98      0.98      0.98      1177
      weighted avg       0.98      0.98      0.98      1177

[[223   0   0   0   0   0]
 [  0 198   7   0   0   1]
 [  0  11 207   0   0   0]
 [  0   0   0 196   3   1]
 [  0   0   0   2 153   3]
 [  0   0   0   0   0 172]]
Accuracy : 0.9762107051826678


In [29]:
for i in range(10, 101, 10):
    cols = fi.sort_values(by='feature_importance_sum')['feature_name'].values[:i]
    temp_x = x_train[cols]
    rf = RandomForestClassifier()
    rf.fit(temp_x, y_train)
    print(f"{i} : {rf.score(x_val[cols], y_val)}")

10 : 0.54035683942226
20 : 0.6015293118096856
30 : 0.6312659303313509
40 : 0.6457094307561597
50 : 0.6508071367884452
60 : 0.6618521665250637
70 : 0.6728971962616822
80 : 0.697536108751062
90 : 0.7179269328802039
100 : 0.7408666100254885


In [ ]:
for i in range(100, 201, 10):
    cols = fi.sort_values(by='feature_importance_sum')['feature_name'].values[:i]
    temp_x = x_train[cols]
    rf = RandomForestClassifier()
    rf.fit(temp_x, y_train)
    print(f"{i} : {rf.score(x_val[cols], y_val)}")

In [ ]:
for i in range(200, 301, 10):
    cols = fi.sort_values(by='feature_importance_sum')['feature_name'].values[:i]
    temp_x = x_train[cols]
    rf = RandomForestClassifier()
    rf.fit(temp_x, y_train)
    print(f"{i} : {rf.score(x_val[cols], y_val)}")

In [ ]:
for i in range(300, 401, 10):
    cols = fi.sort_values(by='feature_importance_sum')['feature_name'].values[:i]
    temp_x = x_train[cols]
    rf = RandomForestClassifier()
    rf.fit(temp_x, y_train)
    print(f"{i} : {rf.score(x_val[cols], y_val)}")

In [ ]:
for i in range(400, 501, 10):
    cols = fi.sort_values(by='feature_importance_sum')['feature_name'].values[:i]
    temp_x = x_train[cols]
    rf = RandomForestClassifier()
    rf.fit(temp_x, y_train)
    print(f"{i} : {rf.score(x_val[cols], y_val)}")

In [30]:
for i in range(500, 551, 10):
    cols = fi.sort_values(by='feature_importance_sum')['feature_name'].values[:i]
    temp_x = x_train[cols]
    rf = RandomForestClassifier()
    rf.fit(temp_x, y_train)
    print(f"{i} : {rf.score(x_val[cols], y_val)}")

500 : 0.9405267629566695
510 : 0.9388275276125744
520 : 0.9371282922684792
530 : 0.9481733220050977
540 : 0.9464740866610025
550 : 0.9498725573491928


### (2) 알고리즘2 :

In [22]:
from sklearn.svm import SVC

model = SVC()
model.fit(x_train, y_train)

y_pred = model.predict(x_val)
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))
print('Accuracy :', accuracy_score(y_val, y_pred))

                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       223
           SITTING       0.94      0.91      0.92       206
          STANDING       0.92      0.94      0.93       218
           WALKING       0.99      1.00      1.00       200
WALKING_DOWNSTAIRS       0.98      0.99      0.99       158
  WALKING_UPSTAIRS       0.99      0.97      0.98       172

          accuracy                           0.97      1177
         macro avg       0.97      0.97      0.97      1177
      weighted avg       0.97      0.97      0.97      1177

[[223   0   0   0   0   0]
 [  1 187  17   0   0   1]
 [  0  13 205   0   0   0]
 [  0   0   0 200   0   0]
 [  0   0   0   0 157   1]
 [  0   0   0   2   3 167]]
Accuracy : 0.967714528462192


### (3) 알고리즘3 :

In [23]:
!pip install xgboost

In [25]:
from xgboost import XGBClassifier
model = XGBClassifier(random_state=42)
model.fit(x_train, y_train)

ValueError: ignored

### (4) 알고리즘4 :

In [20]:
!pip install lightgbm

In [27]:
x_train.values

array([[ 0.27386865, -0.01145928, -0.1538041 , ..., -0.60734083,
         0.36007886,  0.08780331],
       [ 0.27474132, -0.01843989, -0.11759907, ..., -0.86651535,
         0.03909735,  0.09543302],
       [ 0.23758535, -0.00909619, -0.09574538, ..., -0.73828916,
         0.23500969, -0.10411868],
       ...,
       [ 0.18757721,  0.02649229,  0.02617325, ..., -0.87045543,
         0.15853291, -0.0516438 ],
       [ 0.2792925 , -0.01073199, -0.09405676, ..., -0.8529494 ,
         0.12453517,  0.12131963],
       [ 0.26235586, -0.02355174, -0.08803864, ...,  0.37457427,
        -0.52978084, -0.44198892]])

In [26]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(random_state=42)
model.fit(x_train.values, y_train.values)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.084030 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 140146
[LightGBM] [Info] Number of data points in the train set: 4704, number of used features: 561
[LightGBM] [Info] Start training from score -1.662702
[LightGBM] [Info] Start training from score -1.739574
[LightGBM] [Info] Start training from score -1.688825
[LightGBM] [Info] Start training from score -1.774060
[LightGBM] [Info] Start training from score -2.005698
[LightGBM] [Info] Start training from score -1.925291
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightG

LGBMClassifier(random_state=42)

In [28]:
y_pred = model.predict(x_val)
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))
print('Accuracy :', accuracy_score(y_val, y_pred))

                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       223
           SITTING       0.97      0.98      0.97       206
          STANDING       0.98      0.97      0.97       218
           WALKING       0.99      0.99      0.99       200
WALKING_DOWNSTAIRS       1.00      0.99      1.00       158
  WALKING_UPSTAIRS       0.99      1.00      1.00       172

          accuracy                           0.99      1177
         macro avg       0.99      0.99      0.99      1177
      weighted avg       0.99      0.99      0.99      1177

[[223   0   0   0   0   0]
 [  0 201   5   0   0   0]
 [  0   7 211   0   0   0]
 [  0   0   0 199   0   1]
 [  0   0   0   1 157   0]
 [  0   0   0   0   0 172]]
Accuracy : 0.9881053525913339
